# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [2]:
# Load pickled data
import pickle

# TODO: Fill this in based on where you saved the training and testing data

training_file = 'data/train.p'
validation_file= 'data/valid.p'
testing_file = 'data/test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [3]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results

# TODO: Number of training examples
n_train = train['features'].shape[0]

# TODO: Number of validation examples
n_validation = valid['features'].shape[0]

# TODO: Number of testing examples.
n_test = test['features'].shape[0]

# TODO: What's the shape of an traffic sign image?
image_shape = train['features'][0].shape

# TODO: How many unique classes/labels there are in the dataset.
n_classes = y_train.max() + 1

print("Number of training examples =", n_train)
print("Number of valid examples =", n_test)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

Number of training examples = 34799
Number of valid examples = 12630
Number of testing examples = 12630
Image data shape = (32, 32, 3)
Number of classes = 43


### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc. 

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections. It can be interesting to look at the distribution of classes in the training, validation and test set. Is the distribution the same? Are there more examples of some classes than others?

In [6]:
### Data exploration visualization code goes here.
### Feel free to use as many code cells as needed.
import matplotlib.pyplot as plt
# Visualizations will be shown in the notebook.
%matplotlib inline

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

Minimally, the image data should be normalized so that the data has mean zero and equal variance. For image data, `(pixel - 128)/ 128` is a quick way to approximately normalize the data and can be used in this project. 

Other pre-processing steps are optional. You can try different techniques to see if it improves performance. 

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import  tensorflow as tf

tf.random.set_seed(42)


In [8]:
### Preprocess the data here. It is required to normalize the data. Other preprocessing steps could include 
### converting to grayscale, etc.
### Feel free to use as many code cells as needed.
data_augmentor = ImageDataGenerator(rotation_range=15,
                                    zoom_range=.15,
                                    width_shift_range=.1,
                                    height_shift_range=.1,
                                    shear_range=.15,
                                    horizontal_flip=False,
                                    vertical_flip=False,
                                    fill_mode='nearest',
                                    brightness_range=[.2, 1],
                                    )

In [9]:
from sklearn.utils import class_weight
import numpy as np
tf.print(y_train)
tf.print(np.unique(y_train))
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights_dict = dict(enumerate(class_weights))
print(class_weights)
print(class_weights_dict)

array([41, 41, 41, ..., 25, 25, 25], dtype=uint8)
array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42], dtype=uint8)
[4.49599483 0.4087268  0.4026264  0.64228498 0.45721981 0.49047216
 2.24799742 0.62734812 0.64228498 0.6130902  0.44959948 0.69169151
 0.42818998 0.42149952 1.17286822 1.49866494 2.24799742 0.81745361
 0.74933247 4.49599483 2.6975969  2.99732989 2.45236082 1.79839793
 3.37199612 0.59946598 1.49866494 3.85370986 1.68599806 3.37199612
 2.07507454 1.17286822 3.85370986 1.3510502  2.24799742 0.74933247
 2.45236082 4.49599483 0.43509627 2.99732989 2.6975969  3.85370986
 3.85370986]
{0: 4.495994832041344, 1: 0.4087268029128494, 2: 0.40262640286937407, 3: 0.6422849760059063, 4: 0.4572198134279333, 5: 0.49047216349541933, 6: 2.247997416020672, 7: 0.6273481160987922, 8: 0.6422849760059063, 9: 0.6130902043692741, 10: 0.44959948320413434, 11: 0

### Model Architecture

In [10]:
from tensorflow.keras import  layers, models
from tensorflow.python.keras.layers.normalization_v2 import BatchNormalization
from tensorflow.python.keras.layers import Flatten, Dense, Dropout
from tensorflow.python.keras.layers.pooling import MaxPooling2D
from tensorflow.python.keras.regularizers import l2

In [11]:
droup_out = .5
l2_rate = .001

In [ ]:
model = models.Sequential()
# Try using SWISH activation
model.add(layers.Conv2D(8, (3, 3),input_shape=image_shape, activation='swish', padding='same', kernel_regularizer=l2(l2_rate)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2))
# model.add(Dropout(droup_out))

model.add(layers.Conv2D(16, (3, 3), activation='swish', padding='same', kernel_regularizer=l2(l2_rate)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2))
# model.add(Dropout(droup_out))

model.add(layers.Conv2D(32, (3, 3), activation='swish', padding='same', kernel_regularizer=l2(l2_rate)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2))
# model.add(Dropout(droup_out))

model.add(Flatten())
model.add(Dense(256, activation='swish', kernel_regularizer=l2(l2_rate)))
model.add(BatchNormalization())
model.add(Dropout(droup_out))

model.add(Dense(128, activation='swish', kernel_regularizer=l2(l2_rate)))
model.add(BatchNormalization())
model.add(Dropout(droup_out))

model.add(Dense(n_classes,
                activation='softmax'
                ))
### Define your architecture here.
### Feel free to use as many code cells as needed.

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 8)         224       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 8)         32        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 16)        1168      
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 16)        64        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 16)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 32)          4

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [1]:
learning_rate = .01
batch_size = 64
epochs = 100

In [14]:
from tensorflow.python.keras.optimizer_v2.adam import Adam
from tensorflow.python.keras.losses import SparseCategoricalCrossentropy

from datetime import datetime
from tensorflow.python.keras.callbacks import TensorBoard

In [15]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.
optimizer = Adam(learning_rate=learning_rate, decay=0.0003)
loss = SparseCategoricalCrossentropy()

model.compile(optimizer=optimizer, metrics=['accuracy'], loss=loss)
# model.compile(optimizer=optimizer, metrics=['accuracy'], loss='categorical_crossentropy')

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)


In [16]:
# Define the Keras TensorBoard callback.
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=logdir)

In [19]:
H = model.fit(data_augmentor.flow(X_train, y_train, batch_size=batch_size),
                        steps_per_epoch=X_train.shape[0]//batch_size,
                        epochs=epochs,
                        class_weight=class_weights_dict,
                        validation_data=(X_valid, y_valid),
                        # use_multiprocessing=True,
              callbacks=[early_stopping, tensorboard_callback]
                        )

model.save('model'+ datetime.now().strftime("%Y%m%d-%H%M%S"))

Epoch 1/100
543/543 [==============================] - 46s 84ms/step - loss: 2.1662 - accuracy: 0.6325 - val_loss: 1.8595 - val_accuracy: 0.7717

INFO:tensorflow:Assets written to: model20200530-211606\assets


---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [18]:
import tensorflow as tf
import tensorflow.python.keras.models as Model
import os

import cv2
import numpy as np

In [19]:
new_model:Model = tf.keras.models.load_model('model20200530-211606/')
new_model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 8)         224       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 8)         32        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 16)        1168      
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 16)        64        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 16)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 32)          4

In [20]:
signs = np.genfromtxt('signnames.csv', delimiter=',', names=True, dtype=[np.int8, 'U50'])['SignName']

image_folder = 'real_images'

images = np.array(
    [cv2.resize(

        cv2.cvtColor(
            cv2.imread(f'{image_folder}/{filename}'),
            cv2.COLOR_BGR2RGB),
        image_shape[:-1], interpolation=cv2.INTER_LINEAR)

        for filename in os.listdir(image_folder)])

### Load the images and plot them here.
### Feel free to use as many code cells as needed.

### Predict the Sign Type for Each Image

In [21]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.

new_model.evaluate(X_train, y_train)

1088/1088 [==============================] - 11s 10ms/step - loss: 0.4826 - accuracy: 0.9690

[0.48262959718704224, 0.9689933657646179]

In [22]:
predictions: np.ndarray = new_model.predict(tf.convert_to_tensor(images), batch_size=batch_size)

### Analyze Performance

In [23]:
### Calculate the accuracy for these 5 new images.
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.

N = 5

top_n_predictions = tf.nn.top_k(predictions, k=N)

print(top_n_predictions.indices)

tf.Tensor(
[[17 14 12 13 42]
 [28 29 30 23 20]
 [18 27 20 26 25]
 [ 2 19  1 21 23]
 [ 1  2  0  5  4]], shape=(5, 5), dtype=int32)


### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tf.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [24]:
print("Probabilities")
print(top_n_predictions.values.numpy())

string_predictions = np.vectorize(lambda index: signs[index])(top_n_predictions.indices.numpy())

print("Signs")
print(string_predictions)

### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

Probabilities
[[1.0000000e+00 2.3318084e-10 4.8418894e-11 1.9215470e-13 6.1569748e-14]
 [9.9998879e-01 7.6870820e-06 1.5769868e-06 7.5917558e-07 5.4752388e-07]
 [9.3527234e-01 2.6114447e-02 1.2851450e-02 1.0025360e-02 8.4213885e-03]
 [4.4889900e-01 2.6078966e-01 8.8326335e-02 7.4462511e-02 3.5986278e-02]
 [9.5935601e-01 3.3567719e-02 4.9211150e-03 1.0481481e-03 1.9911368e-04]]
Signs
[['No entry' 'Stop' 'Priority road' 'Yield'
  'End of no passing by vehicles over 3.5 metric tons']
 ['Children crossing' 'Bicycles crossing' 'Beware of ice/snow'
  'Slippery road' 'Dangerous curve to the right']
 ['General caution' 'Pedestrians' 'Dangerous curve to the right'
  'Traffic signals' 'Road work']
 ['Speed limit (50km/h)' 'Dangerous curve to the left'
  'Speed limit (30km/h)' 'Double curve' 'Slippery road']
 ['Speed limit (30km/h)' 'Speed limit (50km/h)' 'Speed limit (20km/h)'
  'Speed limit (80km/h)' 'Speed limit (70km/h)']]


### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

---

## Step 4 (Optional): Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [20]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")